In [2]:
import polars as pl

oracle_data = pl.scan_parquet(
    "s3://covid-variant-nowcast-hub/target-data/oracle-output/*/*.parquet",
    storage_options={"skip_signature": "true"}
)

# filter and transform as needed and collect into a dataframe, for example:
oracle_dataframe = oracle_data.filter(pl.col("location") == "MA").collect()
oracle_dataframe.head()

/var/folders/15/gjw1xqz91mq9q73dqlw93_780000gn/T/ipykernel_52674/3417216582.py:9: UserWarning: '(default_)region' not set; polars will try to get it from bucket

Set the region manually to silence this warning.
  oracle_dataframe = oracle_data.filter(pl.col("location") == "MA").collect()


location,target_date,clade,oracle_value,nowcast_date
str,date,str,f64,date
"""MA""",2024-09-08,"""24A""",1.0,2024-10-09
"""MA""",2024-09-08,"""24B""",0.0,2024-10-09
"""MA""",2024-09-08,"""24C""",0.0,2024-10-09
"""MA""",2024-09-08,"""24E""",0.0,2024-10-09
"""MA""",2024-09-08,"""recombinant""",0.0,2024-10-09


In [3]:
import polars as pl

# create a LazyFrame for model-output files
lf = pl.scan_parquet(
    "s3://covid-variant-nowcast-hub/model-output/*/*.parquet",
    storage_options={"skip_signature": "true"}
)

# use a collect operation to materialize the LazyFrame into a DataFrame
model_output = lf.collect(streaming=True)

/var/folders/15/gjw1xqz91mq9q73dqlw93_780000gn/T/ipykernel_52674/2167261014.py:10: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  model_output = lf.collect(streaming=True)


In [4]:
model_output.describe()

statistic,nowcast_date,target_date,location,clade,output_type,output_type_id,value,round_id,model_id
str,str,str,str,str,str,str,f64,str,str
"""count""","""66877860""","""66877860""","""66877860""","""66877860""","""66877860""","""66288600""",6.687786e7,"""66877860""","""66877860"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""589260""",0.0,"""0""","""0"""
"""mean""","""2024-12-21 07:42:37.627000""","""2024-12-10 19:42:37.627000""",null,null,null,null,0.108798,null,null
"""std""",null,null,null,null,null,null,0.190755,null,null
"""min""","""2024-10-09""","""2024-09-08""","""AK""","""24A""","""mean""","""AK00""",0.0,"""2024-10-09""","""CADPH-CATaLog"""
"""25%""","""2024-11-20""","""2024-11-11""",null,null,null,null,0.003446,null,null
"""50%""","""2024-12-25""","""2024-12-11""",null,null,null,null,0.021906,null,null
"""75%""","""2025-01-22""","""2025-01-11""",null,null,null,null,0.090286,null,null
"""max""","""2025-02-19""","""2025-03-01""","""WY""","""recombinant""","""sample""","""WY99""",1.0,"""2025-02-19""","""UMass-HMLR"""


In [6]:
type(model_output)

polars.dataframe.frame.DataFrame

In [7]:
model_output.unique(subset=["model_id"], maintain_order=True)

nowcast_date,target_date,location,clade,output_type,output_type_id,value,round_id,model_id
date,date,str,str,str,str,f64,str,str
2024-10-30,2024-09-29,"""CA""","""24A""","""mean""",null,0.148496,"""2024-10-30""","""CADPH-CATaLog"""
2024-10-23,2024-09-22,"""AL""","""24E""","""mean""",null,0.0,"""2024-10-23""","""LANL-CovTransformer"""
2024-10-30,2024-09-29,"""AL""","""24A""","""sample""","""AL1""",0.197177,"""2024-10-30""","""UGA-multicast"""
2024-10-09,2024-09-08,"""AL""","""24A""","""sample""","""AL00""",0.008923,"""2024-10-09""","""UMass-HMLR"""
